**World of Warships**

Project work per la ricerca di un sistema di valutazione dei giocatori sul gioco World of warships, tramite l'analisi dei dati ricavati tramite
le API pubbliche fornite dagli sviluppatori del gioco.


Step - 1

Importate librerie per pandas e numpy.

Creazione di una funzione per cercare tutti i file json all'interno della cartella 'accounts'
e inserimento dei dati all'interno di un dataframe.

Dopo la creazione del dataframe, vengono eliminate le colonne non rilevanti al nostro scopo.

Infine viene creato un file csv contenente il nostro dataframe pulito.

In [46]:
import pandas as pd
import numpy as np
import json
import re,os,glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

path_to_json = 'accounts/'
json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)

dfs = []

for file in file_list:
    data = open(file).read()
    df=pd.json_normalize(json.loads(data))
    df.columns = [re.sub(r'data\..+\.', '', col_name) for col_name in df.columns]
    dfs.append(df)

dfaccounts = pd.concat(dfs, ignore_index=True)
dfaccounts.columns
dfaccounts_completed = dfaccounts.drop(columns=['ships', 'statistics.pvp.losses', 'stats_updated_at', 'statistics.pvp.capture_points', 'statistics.pvp.team_capture_points', 'statistics.pvp.team_dropped_capture_points','statistics.pvp.ramming.frags', 'statistics.pvp.frags', 'statistics.distance', 'statistics.pvp.main_battery.frags', 'statistics.pvp.second_battery.frags', 'statistics.pvp.torpedoes.frags', 'statistics.pvp.aircraft.frags', 'statistics.pvp.art_agro', 'statistics.pvp.torpedo_agro','last_battle_time', 'leveling_tier', 'created_at', 'leveling_points', 'updated_at', 'private', 'hidden_profile', 'logout_at', 'karma', 'statistics.pvp.max_xp', 'statistics.pvp.damage_to_buildings', 'statistics.pvp.main_battery.max_frags_battle', 'statistics.pvp.main_battery.max_frags_ship_id', 'statistics.pvp.max_ships_spotted_ship_id','statistics.pvp.max_damage_scouting', 'statistics.pvp.max_xp_ship_id', 'statistics.pvp.second_battery.max_frags_battle', 'statistics.pvp.second_battery.max_frags_ship_id', 'statistics.pvp.max_frags_ship_id', 'statistics.pvp.xp', 'statistics.pvp.dropped_capture_points', 'statistics.pvp.max_damage_dealt_to_buildings', 'statistics.pvp.draws', 'statistics.pvp.battles_since_510', 'statistics.pvp.max_total_agro_ship_id', 'statistics.pvp.battles', 'statistics.pvp.max_ships_spotted', 'statistics.pvp.max_suppressions_ship_id', 'statistics.pvp.max_total_agro', 'statistics.pvp.max_frags_battle', 'statistics.pvp.ramming.max_frags_battle', 'statistics.pvp.ramming.max_frags_ship_id', 'statistics.pvp.suppressions_count', 'statistics.pvp.max_suppressions_count', 'statistics.pvp.torpedoes.max_frags_battle', 'statistics.pvp.torpedoes.max_frags_ship_id', 'statistics.pvp.max_planes_killed_ship_id', 'statistics.pvp.aircraft.max_frags_battle', 'statistics.pvp.aircraft.max_frags_ship_id', 'statistics.pvp.max_damage_dealt', 'statistics.pvp.max_damage_dealt_to_buildings_ship_id', 'statistics.pvp.max_damage_dealt_ship_id', 'statistics.pvp.damage_dealt', 'statistics.pvp.max_planes_killed', 'statistics.pvp.max_scouting_damage_ship_id', 'statistics.pvp.battles_since_512', 'nickname'], axis=1)

dfaccounts_completed.head(20)

dfaccounts_completed.to_csv('accounts.csv')

STEP - 2

Lettura del file csv e pulizia dei nomi delle colonne tramite la rimozione di statistics.pvp

Riduzione delle colonne al fine di eliminare differenze sul tempo giocato che risulta essere differente per ogni giocatore., tramite la normalizzazione dei dati relativi ai colpi sparati e colpiti, vittorie, e partite in cui il giocatore è sopravvissuto fino al termine.

Eliminazione delle precedenti colonne perchè non più significative e di ostacolo per i futuri calcoli.


In [47]:
df = pd.read_csv('accounts.csv',index_col=[0])
df.columns = df.columns.str.replace('statistics.', '')
df.columns = df.columns.str.replace('pvp.', '')
df.columns = df.columns.str.replace('.', '_')

df['main_precision'] = ((df['main_battery_hits'] / df['main_battery_shots']) * 100)
df['second_precision'] = ((df['second_battery_hits'] / df['second_battery_shots']) * 100)
df['torpedoes_precision'] = ((df['torpedoes_hits'] / df['torpedoes_shots']) * 100)
df['ships_spotted_precision'] = ((df['ships_spotted'] / df['battles']) * 100)
df['survived_battles_precision'] = ((df['survived_battles'] / df['battles']) * 100)
df['planes_killed_precision'] = ((df['planes_killed'] / df['battles']) * 100)
df['survived_wins_precision'] = ((df['survived_wins'] / df['battles']) * 100)
df['wins_precision'] = ((df['wins'] / df['battles']) * 100)
new_df = df.drop(columns=['control_captured_points', 'control_dropped_points','battles' ,'main_battery_shots','second_battery_shots', 'torpedoes_shots' ,'main_battery_hits', 'second_battery_hits', 'torpedoes_hits', 'ships_spotted', 'survived_battles', 'control_captured_points', 'planes_killed', 'survived_wins', 'control_dropped_points', 'wins'])
new_df.head(20)

,account_id,damage_scouting,main_precision,second_precision,torpedoes_precision,ships_spotted_precision,survived_battles_precision,planes_killed_precision,survived_wins_precision,wins_precision
0,578060406,37803969,35.218308,13.956126,7.502988,46.485664,28.647551,157.472723,22.126364,34.331388
1,578887227,47247482,32.847889,20.859793,4.028882,9.829661,5.886336,32.525916,4.334011,6.349951
2,551869913,9426181,26.422118,13.236634,8.485981,39.661799,21.752498,94.235204,17.601845,42.890085
3,501529911,13357862,30.008562,13.532738,14.378102,12.518055,24.169475,96.822340,17.621570,31.535869
4,500392101,138109496,27.603193,15.229768,3.610605,72.545063,43.570083,211.689535,29.149852,43.126177
5,512738038,142304545,32.194120,13.278144,5.345886,65.094149,19.244786,111.550921,17.392185,41.516501
6,512941313,82264756,35.123101,14.250277,5.682968,65.743488,21.958457,202.192549,19.090010,41.411144
7,569690009,155427264,29.964250,18.511102,5.521623,35.165615,26.940063,197.539432,21.561514,38.201893
8,528473841,79939952,33.977640,17.722434,7.283484,50.784593,52.335949,279.083452,37.107703,48.716120
9,564409769,88522204,26.663280,13.101441,3.266274,27.711640,16.197090,97.962963,11.951058,18.253968


STEP - 3

Creazione di una colonna 'index' che conterrà l'indice di valutazione dei giocatori.
L'indice viene creato dando un peso diverso alle varie precisioni ottenute precedentemente.
E' stato optato per un forte peso sulla precisione dei colpi sparati dai vari equipaggiamenti a disposione delle navi (20), sono stati dati valori minori per le colonne che risultavano secondarie rispetto alle altre.

In [34]:
new_df['index'] = (df['main_precision'] * 20) + (df['second_precision'] *20) + (df['torpedoes_precision'] * 20) + (df['ships_spotted_precision'] * 7) + (df['survived_battles_precision'] *10) + (df['planes_killed_precision'] * 1.5) + (df['survived_wins_precision'] *7) + (df['wins_precision'] * 5)
df_accounts = new_df.drop(columns=['main_precision', 'second_precision', 'torpedoes_precision', 'ships_spotted_precision', 'survived_battles_precision', 'planes_killed_precision', 'survived_wins_precision', 'wins_precision'])
df_accounts.sort_values(by=['index'],ascending=False).head(10)

,account_id,damage_scouting,index
11,517713114,233294628,3279.313506
29,530145684,320928227,3128.188262
32,500004385,385613787,3001.155604
8,528473841,79939952,2980.482505
14,673416937,523407,2817.469858
25,528832776,63654473,2672.775291
21,533573166,71930563,2643.758057
4,500392101,138109496,2609.601746
16,515240356,58835464,2600.721869
40,512159521,243088950,2591.940404
